# WRC Live Timing API Scraper

Simple exploration of grabbing data from WRC Live Timing API.

TO DO - how do we lookup `championshipId`?

In [283]:
from urllib.parse import urljoin
from parse import parse
import pathlib

import pandas as pd
import requests

#%pip install requests-cache
import requests_cache
requests_cache.install_cache("demo_cache2")

In [322]:
CURRENT_YEAR = 2023

WRC_API_BASE = "https://api.wrc.com/content/{path}"

In [323]:
from itertools import zip_longest


def _WRC_json(path, base=WRC_API_BASE, retUrl=False):
    """Return JSON from API."""
    url = urljoin(base, path)
    if retUrl:
        return url
    # print(f"Fetching: {url}")
    r = requests.get(url)
    rj = r.json()
    if "status" in rj and rj["status"]=="Not Found":
        return {}
    return r.json()


def convert_date_range(date_range_str):
    """Convert date of from `19 - 22 JAN 2023` to date range."""
    r = parse("{start_day} - {end_day} {month} {year}", date_range_str)
    start_date = pd.to_datetime(
        f"{r['start_day']} {r['month']} {r['year']}", format="%d %b %Y"
    )
    end_date = pd.to_datetime(
        f"{r['end_day']} {r['month']} {r['year']}", format="%d %b %Y"
    )
    return pd.date_range(start=start_date, end=end_date)


def timeify(df, col, typ=None):
    """Convert a column  to a datetime inplace."""
    if typ == "daterange":
        df[col] = df[col].apply(convert_date_range)
    else:
        df[col] = pd.to_datetime(df[col].astype(int), unit="ms")


def tablify(json_data, subcolkey=None):
    """Generate table from separate colnames/values JSON."""
    # Note that the JSON may be a few rows short cf. provided keys
    if subcolkey is None:
        results = []
        for values in json_data["values"]:
            result_dict = {}
            # Zip keys and values, filling None for missing values
            zipped_values = zip_longest(json_data["fields"], values, fillvalue=None)
            # Convert the zipped values to a dictionary and update the result_dict
            result_dict.update(dict(zipped_values))
            results.append(result_dict)
        return pd.DataFrame(results)
    else:
        df = pd.DataFrame(columns=json_data["fields"])
        for value in json_data["values"]:
            _df = pd.DataFrame(value[subcolkey])
            if len(_df.columns) < len(json_data["fields"]):
                _df[[json_data["fields"][len(_df.columns) :]]] = None
            _df.columns = json_data["fields"]
            for c in [k for k in value.keys() if k != subcolkey]:
                _df[c] = value[c]
                df = pd.concat([df, _df])
        return df

def stage_id_annotations(df, eventId=None, rallyId=None, stageId=None):
    if "eventId" not in df.columns:
        df["eventId"] = eventId
    if "rallyId" not in df.columns:
        df["rallyId"] = rallyId
    if "stageId" not in df.columns:
        df["stageId"] = stageId

## Season Calendar

The API call `https://api.wrc.com/content/filters/calendar` gives a list of current or latest (last, if season's end) events, from which we can get a season identifer. For the 2023 WRC championship, the season ID is `20`

In [324]:
"""
https://www.wrc.com/live-timing?liveTimingMenu=overall_livetiming&stage=FINAL&championshipId=249
<div id="dropdown" style="max-height: 92.25px;">
<div data-key="245" class="sc-dVhcbM hzWUOI">ALL</div>
<div data-key="247" class="sc-dVhcbM hzWUOI">WRC</div>
<div data-key="249" class="sc-dVhcbM hzWUOI" active="true">WRC2</div>
<div data-key="256" class="sc-dVhcbM hzWUOI">WRC3</div>
<div data-key="258" class="sc-dVhcbM hzWUOI">Junior WRC</div>
<div data-key="251" class="sc-dVhcbM hzWUOI">WRC2 Challenger</div>
<div data-key="254" class="sc-dVhcbM hzWUOI">Master Cup</div></div>

"""

championshipIds = {"all": 245,
                   "wrc": 247,
                   "wrc2": 249,
                   "wrc3": 256,
                   "jwrc": 258,
                   "wrc2c": 251,
                   "mcup": 254}

championshipId = championshipIds["wrc"]

In [325]:
stub = f"filters/calendar?language=en&size=20&championship=wrc&origin=vcms&year=2024"
json_data = _WRC_json(stub)
keys_to_keep = ['id', 'guid', 'title', 'location', 'startDate', 'endDate', 'eventId',
       'rallyId', 'description', 'round', 'cvpSeriesLink', 'sponsor', 'images',
       'season', 'competition', 'country', 'asset', '__typename', 'type',
       'uid', 'seriesUid', 'releaseYear', 'availableOn', 'availableTill',
       'startDateLocal', 'endDateLocal', 'finishDate', 'championship',
       'championshipLogo'
]
filtered_data = [
    {key: item[key] for key in keys_to_keep if key in item}
    for item in json_data["content"]
]

pd.DataFrame(filtered_data)

,id,guid,title,location,startDate,endDate,eventId,rallyId,description,round,...,uid,seriesUid,releaseYear,availableOn,availableTill,startDateLocal,endDateLocal,finishDate,championship,championshipLogo
0,C87D,WRC_2024_01,Rallye Monte-Carlo,Monaco,1706110200000,1706454000000,446,477,The most unpredictable rally of the year. Rela...,1,...,C87D,WRC_2024_01,2024,1706110200000,1706454000000,2024-01-24T16:30:00+01:00,2024-01-28T16:00:00+01:00,1706454000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
1,ANhw,WRC_2024_02,WRC Rally Sweden,sweden,1707984000000,1708268400000,447,478,The WRC’s ultimate winter challenge. Watch in ...,2,...,ANhw,WRC_2024_02,2024,1707984000000,1708268400000,2024-02-15T09:00:00+01:00,2024-02-18T16:00:00+01:00,1708268400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
2,tv2t,WRC_2024_03,WRC Safari Rally Kenya,"Naivasha, Kenya",1711522800000,1711890000000,448,479,Held on the untamed terrains of Africa’s breat...,3,...,tv2t,WRC_2024_03,2024,1711522800000,1711890000000,2024-03-27T10:00:00+03:00,2024-03-31T16:00:00+03:00,1711890000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
3,JCpO,WRC_2024_04,WRC Croatia Rally,"Zagreb, City of Zagreb Region",1713423600000,1713708000000,449,480,High-speed stretches meet challenging hairpin ...,4,...,JCpO,WRC_2024_04,2024,1713423600000,1713708000000,2024-04-18T09:00:00+02:00,2024-04-21T16:00:00+02:00,1713708000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
4,lLrv,WRC_2024_05,WRC Vodafone Rally de Portugal,"Matosinhos, Porto",1715238000000,1715522400000,450,481,Fast but technical gravel roads inland from Po...,5,...,lLrv,WRC_2024_05,2024,1715238000000,1715522400000,2024-05-09T08:00:00+01:00,2024-05-12T15:00:00+01:00,1715522400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
5,5VRr,WRC_2024_06,WRC Rally Italia Sardegna,"Olbia, Sardinia",1717087500000,1717329600000,452,483,Don’t let the picturesque Mediterranean backdr...,6,...,5VRr,WRC_2024_06,2024,1717087500000,1717329600000,2024-05-30T18:45:00+02:00,2024-06-02T14:00:00+02:00,1717329600000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
6,htOR,WRC_2024_07,WRC ORLEN 80th Rally Poland,Poland,1719475200000,1719756000000,453,484,Back on the WRC calendar! Poland's lightning-f...,7,...,htOR,WRC_2024_07,2024,1719475200000,1719756000000,2024-06-27T10:00:00+02:00,2024-06-30T16:00:00+02:00,1719756000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
7,228Z,WRC_2024_08,WRC Tet Rally Latvia,"Liepāja, Latvia",1721277060000,1721566800000,454,485,WRC is thrilled to welcome a newcomer in Tet R...,8,...,228Z,WRC_2024_08,2024,1721277060000,1721566800000,2024-07-18T07:31:00+03:00,2024-07-21T16:00:00+03:00,1721566800000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
8,Dv15,WRC_2024_09,WRC Secto Rally Finland,Jyväskylä,1722441600000,1722772800000,455,486,A mecca for rally drivers and enthusiasts alik...,9,...,Dv15,WRC_2024_09,2024,1722441600000,1722772800000,2024-07-31T19:00:00+03:00,2024-08-04T15:00:00+03:00,1722772800000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
9,csx1,WRC_2024_10,WRC EKO Acropolis Rally Greece,Lamia,1725516060000,1725796800000,456,487,A legendary WRC fixture. Twisty gravel mountai...,10,...,csx1,WRC_2024_10,2024,1725516060000,1725796800000,2024-09-05T09:01:00+03:00,2024-09-08T15:00:00+03:00,1725796800000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...


In [326]:
def getFullCalendar(year=CURRENT_YEAR, championship="wrc", size=20):
    stub = f"filters/calendar?language=en&size={size}&championship={championship}&origin=vcms&year={year}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.dataFrame()
    # return json_data
    return pd.DataFrame(json_data["content"])


df_fullcal = getFullCalendar()
df_fullcal.head()

,id,guid,title,location,startDate,endDate,eventId,rallyId,description,round,...,uid,seriesUid,releaseYear,availableOn,availableTill,startDateLocal,endDateLocal,finishDate,championship,championshipLogo
0,IQXg,WRC_2023_01,Rallye Monte-Carlo,Monaco,1674125280000,1674384480000,353,357,The most unpredictable rally of the year. Rela...,1,...,IQXg,WRC_2023_01,2023,1674125280000,1674384480000,2023-01-19T11:48:00+01:00,2023-01-22T11:48:00+01:00,1674384480000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
1,lGzi,WRC_2023_02,WRC Rally Sweden,Umea,1675940100000,1676199300000,354,358,The WRC’s ultimate winter challenge. Watch in ...,2,...,lGzi,WRC_2023_02,2023,1675940100000,1676199300000,2023-02-09T11:55:00+01:00,2023-02-12T11:55:00+01:00,1676199300000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
2,siDN,WRC_2023_03,WRC Guanajuato Rally México,"Leon, Guanajuato",1678949100000,1679208300000,355,359,A thrilling ride through the colourful landsca...,3,...,siDN,WRC_2023_03,2023,1678949100000,1679208300000,2023-03-16T00:45:00-06:00,2023-03-19T00:45:00-06:00,1679208300000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
3,rejA,WRC_2023_04,WRC Croatia Rally,"Zagreb, City of Zagreb Region",1682059680000,1682232480000,356,360,High-speed stretches meet challenging hairpin ...,4,...,rejA,WRC_2023_04,2023,1682059680000,1682232480000,2023-04-21T08:48:00+02:00,2023-04-23T08:48:00+02:00,1682232480000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
4,9xIl,WRC_2023_05,WRC Vodafone Rally de Portugal,"Matosinhos, Porto",1683870600000,1684043400000,357,361,Fast but technical gravel roads inland from Po...,5,...,9xIl,WRC_2023_05,2023,1683870600000,1684043400000,2023-05-12T06:50:00+01:00,2023-05-14T06:50:00+01:00,1684043400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...


In [327]:
len(df_fullcal)

13

In [328]:
df_fullcal.columns

Index(['id', 'guid', 'title', 'location', 'startDate', 'endDate', 'eventId',
       'rallyId', 'description', 'round', 'cvpSeriesLink', 'sponsor', 'images',
       'season', 'competition', 'country', 'asset', '__typename', 'type',
       'uid', 'seriesUid', 'releaseYear', 'availableOn', 'availableTill',
       'startDateLocal', 'endDateLocal', 'finishDate', 'championship',
       'championshipLogo'],
      dtype='object')

In [329]:
df_fullcal

,id,guid,title,location,startDate,endDate,eventId,rallyId,description,round,...,uid,seriesUid,releaseYear,availableOn,availableTill,startDateLocal,endDateLocal,finishDate,championship,championshipLogo
0,IQXg,WRC_2023_01,Rallye Monte-Carlo,Monaco,1674125280000,1674384480000,353,357,The most unpredictable rally of the year. Rela...,1,...,IQXg,WRC_2023_01,2023,1674125280000,1674384480000,2023-01-19T11:48:00+01:00,2023-01-22T11:48:00+01:00,1674384480000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
1,lGzi,WRC_2023_02,WRC Rally Sweden,Umea,1675940100000,1676199300000,354,358,The WRC’s ultimate winter challenge. Watch in ...,2,...,lGzi,WRC_2023_02,2023,1675940100000,1676199300000,2023-02-09T11:55:00+01:00,2023-02-12T11:55:00+01:00,1676199300000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
2,siDN,WRC_2023_03,WRC Guanajuato Rally México,"Leon, Guanajuato",1678949100000,1679208300000,355,359,A thrilling ride through the colourful landsca...,3,...,siDN,WRC_2023_03,2023,1678949100000,1679208300000,2023-03-16T00:45:00-06:00,2023-03-19T00:45:00-06:00,1679208300000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
3,rejA,WRC_2023_04,WRC Croatia Rally,"Zagreb, City of Zagreb Region",1682059680000,1682232480000,356,360,High-speed stretches meet challenging hairpin ...,4,...,rejA,WRC_2023_04,2023,1682059680000,1682232480000,2023-04-21T08:48:00+02:00,2023-04-23T08:48:00+02:00,1682232480000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
4,9xIl,WRC_2023_05,WRC Vodafone Rally de Portugal,"Matosinhos, Porto",1683870600000,1684043400000,357,361,Fast but technical gravel roads inland from Po...,5,...,9xIl,WRC_2023_05,2023,1683870600000,1684043400000,2023-05-12T06:50:00+01:00,2023-05-14T06:50:00+01:00,1684043400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
5,BTVM,WRC_2023_06,WRC Rally Italia Sardegna,"Olbia, Sardinia",1685602260000,1685861460000,358,362,Don’t let the picturesque Mediterranean backdr...,6,...,BTVM,WRC_2023_06,2023,1685602260000,1685861460000,2023-06-01T08:51:00+02:00,2023-06-04T08:51:00+02:00,1685861460000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
6,vhG4,WRC_2023_07,WRC Safari Rally Kenya,"Naivasha, Kenya",1687416720000,1687675920000,359,363,Held on the untamed terrains of Africa’s breat...,7,...,vhG4,WRC_2023_07,2023,1687416720000,1687675920000,2023-06-22T09:52:00+03:00,2023-06-25T09:52:00+03:00,1687675920000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
7,38Ly,WRC_2023_08,WRC Rally Estonia,Tartu,1689836040000,1690095240000,360,364,Mind-bendingly fast smooth gravel stages inclu...,8,...,38Ly,WRC_2023_08,2023,1689836040000,1690095240000,2023-07-20T09:54:00+03:00,2023-07-23T09:54:00+03:00,1690095240000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
8,Kf3P,WRC_2023_09,WRC Secto Automotive Rally Finland,Jyväskylä,1691001000000,1691346540000,361,365,A mecca for rally drivers and enthusiasts alik...,9,...,Kf3P,WRC_2023_09,2023,1691001000000,1691346540000,2023-08-02T21:30:00+03:00,2023-08-06T21:29:00+03:00,1691346540000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
9,aCfi,WRC_2023_10,WRC EKO Acropolis Rally Greece,Lamia,1694097840000,1694346300000,362,366,A legendary WRC fixture. Twisty gravel mountai...,10,...,aCfi,WRC_2023_10,2023,1694097840000,1694346300000,2023-09-07T17:44:00+03:00,2023-09-10T14:45:00+03:00,1694346300000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...


Let's get the `seasonId` from a season record in the `season` column:

In [330]:
df_fullcal["season"][0]

{'id': 'c0119c1c-bda7-403f-aa4d-29ae30959caa',
 'name': None,
 'seasonId': '20',
 'title': '2023',
 '__typename': 'Season'}

In [331]:
CURRENT_SEASON = df_fullcal["season"][0]["seasonId"]

seasonId = CURRENT_SEASON

# Check
# assert int(CURRENT_SEASON)==28 if CURRENT_YEAR==2024 else False
# And / or check the title?
# assert df_fullcal["season"][0]['title'] == '2024'

CURRENT_SEASON

'20'

In [332]:
# https://api.wrc.com/content/result/calendar?season=20&championship=wrc

def getResultsCalendar(seasonId=CURRENT_SEASON,  retUrl=False):
    """Get the WRC Calendar for a given season ID as a JSON result."""
    stub = f"result/calendar?season={seasonId}&championship=wrc"
    if retUrl:
        return stub
    json_data = _WRC_json(stub)
    df_calendar = tablify(json_data)
    #timeify(df_calendar, "date", "daterange")
    #timeify(df_calendar, "startDate")
    #timeify(df_calendar, "finishDate")
    # df_calendar.set_index("id", inplace=True)
    return df_calendar

In [333]:
getResultsCalendar().columns

Index(['id', 'rallyTitle', 'ROUND', 'rallyCountry', 'rallyCountryImage',
       'rallyId', 'date', 'startDate', 'finishDate', 'driverId',
       'driverCountryImage', 'driver', 'coDriverId', 'coDriverCountryImage',
       'coDriver', 'teamId', 'teamLogo', 'teamName', 'manufacturer'],
      dtype='object')

In [334]:
getResultsCalendar(retUrl=True)

'result/calendar?season=20&championship=wrc'

In [335]:
getResultsCalendar().head()

,id,rallyTitle,ROUND,rallyCountry,rallyCountryImage,rallyId,date,startDate,finishDate,driverId,driverCountryImage,driver,coDriverId,coDriverCountryImage,coDriver,teamId,teamLogo,teamName,manufacturer
0,IQXg,Rallye Monte-Carlo,1,Monaco,Flags/MCO.png,357,19 - 22 JAN 2023,1674125280000,1674384480000,1cf9ade6-25b5-5586-8393-eacfeb943eae,Flags/FRA.png,Sébastien OGIER,867cd58f-3fe3-5290-9cc5-b21cb41c523c,Flags/FRA.png,Vincent LANDAIS,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Flags/toyota.png,TOYOTA GAZOO RACING WRT,Toyota
1,lGzi,WRC Rally Sweden,2,Sweden,Flags/SWE.png,358,09 - 12 FEB 2023,1675940100000,1676199300000,6632e7ca-34bf-55b8-9cad-d060000fa794,Flags/EST.png,Ott TÄNAK,00a8a5c3-f7ba-5086-86df-1a59b7da7e26,Flags/EST.png,Martin JÄRVEOJA,887309d0-48be-5f83-ad3e-abf2a79a64a3,Flags/ford.png,M-SPORT FORD WORLD RALLY TEAM,Ford
2,siDN,WRC Guanajuato Rally México,3,Mexico,Flags/MEX.png,359,16 - 19 MAR 2023,1678949100000,1679208300000,1cf9ade6-25b5-5586-8393-eacfeb943eae,Flags/FRA.png,Sébastien OGIER,867cd58f-3fe3-5290-9cc5-b21cb41c523c,Flags/FRA.png,Vincent LANDAIS,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Flags/toyota.png,TOYOTA GAZOO RACING WRT,Toyota
3,rejA,WRC Croatia Rally,4,Croatia,Flags/HRV.png,360,21 - 23 APR 2023,1682059680000,1682232480000,ae7329c9-79b3-5d96-886c-22cca6217764,Flags/GBR.png,Elfyn EVANS,53e56691-fe7c-5271-9dc5-8960df28b221,Flags/GBR.png,Scott MARTIN,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Flags/toyota.png,TOYOTA GAZOO RACING WRT,Toyota
4,9xIl,WRC Vodafone Rally de Portugal,5,Portugal,Flags/PRT.png,361,12 - 14 MAY 2023,1683870600000,1684043400000,d8e4bbea-3af2-5486-9ad5-a445aaec573e,Flags/FIN.png,Kalle ROVANPERÄ,e4dd8a3f-00e9-59f7-9871-9337af6085d7,Flags/FIN.png,Jonne HALTTUNEN,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Flags/toyota.png,TOYOTA GAZOO RACING WRT,Toyota


In [336]:
getResultsCalendar().tail()

,id,rallyTitle,ROUND,rallyCountry,rallyCountryImage,rallyId,date,startDate,finishDate,driverId,driverCountryImage,driver,coDriverId,coDriverCountryImage,coDriver,teamId,teamLogo,teamName,manufacturer
8,Kf3P,WRC Secto Automotive Rally Finland,9,Finland,Flags/FIN.png,365,02 - 06 AUG 2023,1691001000000,1691346540000,ae7329c9-79b3-5d96-886c-22cca6217764,Flags/GBR.png,Elfyn EVANS,53e56691-fe7c-5271-9dc5-8960df28b221,Flags/GBR.png,Scott MARTIN,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Flags/toyota.png,TOYOTA GAZOO RACING WRT,Toyota
9,aCfi,WRC EKO Acropolis Rally Greece,10,Greece,Flags/GRC.png,366,07 - 10 SEP 2023,1694097840000,1694346300000,d8e4bbea-3af2-5486-9ad5-a445aaec573e,Flags/FIN.png,Kalle ROVANPERÄ,e4dd8a3f-00e9-59f7-9871-9337af6085d7,Flags/FIN.png,Jonne HALTTUNEN,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Flags/toyota.png,TOYOTA GAZOO RACING WRT,Toyota
10,ayMP,WRC Rally Chile Bio Bío,11,Chile,Flags/CHL.png,367,28 - 01 OCT 2023,1695898800000,1696182300000,6632e7ca-34bf-55b8-9cad-d060000fa794,Flags/EST.png,Ott TÄNAK,00a8a5c3-f7ba-5086-86df-1a59b7da7e26,Flags/EST.png,Martin JÄRVEOJA,887309d0-48be-5f83-ad3e-abf2a79a64a3,Flags/ford.png,M-SPORT FORD WORLD RALLY TEAM,Ford
11,dtcK,WRC Central European Rally,12,Europe,Flags/EUR.png,368,25 - 29 OCT 2023,1698242400000,1698588000000,c99a2a26-bd03-5153-aaa7-684d3acb5491,Flags/BEL.png,Thierry NEUVILLE,b1b98699-0332-528a-8a80-11eb538f1ded,Flags/BEL.png,Martijn WYDAEGHE,b6692ea5-df92-5cad-a91c-20319a6fffd7,Flags/hyundai.png,HYUNDAI SHELL MOBIS WORLD RALLY TEAM,Hyundai
12,k7il,WRC FORUM8 Rally Japan,13,Japan,Flags/JPN.png,369,16 - 19 NOV 2023,1700092800000,1700377260000,ae7329c9-79b3-5d96-886c-22cca6217764,Flags/GBR.png,Elfyn EVANS,53e56691-fe7c-5271-9dc5-8960df28b221,Flags/GBR.png,Scott MARTIN,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Flags/toyota.png,TOYOTA GAZOO RACING WRT,Toyota


In [337]:
import datetime

def timeNow(typ="ms"):
    now = int(datetime.datetime.now().timestamp())
    if typ=="ms":
        now *=1000
    return now

timeNow()

1732733153000

In [338]:
eventId, rallyId = df_fullcal[df_fullcal["startDate"] < timeNow()].iloc[-1][
    ["eventId", "rallyId"]
]
eventId, rallyId

('365', '369')

In [339]:
## Stages


def getStageDetails(eventId, rallyId):
    stub = f"result/stages?eventId={eventId}&rallyId={rallyId}&championship=wrc"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.dataFrame()
    df_stageDetails = tablify(json_data)
    return df_stageDetails


df_stageDetails = getStageDetails(eventId, rallyId)
df_stageDetails.head(3)

,id,STAGE,STAGE TYPE,stageId,eventId,STATUS,day,name,distance
0,ee55daa1-fd07-574f-835a-f39008081b95,SHD,shakedown,SHD,365,Complete,,Shakedown,
1,3cd7a38a-436f-5c21-b966-8b4743f20a57,SS1,HeadToHeadSuperSpecialStage,6119,365,Completed,Thursday,SS1 TOYOTA STADIUM SSS 1 (2.1km),2.1
2,7ae2243b-6746-516b-895e-16b5d0a2ed51,SS2,SpecialStage,6120,365,Interrupted,Friday,SS2 Isegami's Tunnel 1 (23.67km),23.67


In [340]:
df_stageDetails.tail()

,id,STAGE,STAGE TYPE,stageId,eventId,STATUS,day,name,distance
19,9529ae88-0d8e-5449-abf9-5e7d43935cc9,SS19,SpecialStage,6137,365,Completed,Sunday,SS19 Nenoue Kougen 1 (11.6km),11.6
20,128c0d05-4538-536f-a7d2-f929836d1dc4,SS20,SpecialStage,6138,365,Completed,Sunday,SS20 Ena City 2 (22.92km),22.92
21,a9e4c371-57ba-5c72-8fff-8b92a95baa60,SS21,SpecialStage,6139,365,Completed,Sunday,SS21 Nenoue Kougen 2 (11.6km),11.6
22,4a0db031-0143-5756-af1f-e87945227001,SS22,PowerStage,6140,365,Completed,Sunday,SS22 Wolf Power Stage Asahi Kougen 2 (7.52km),7.52
23,f8f13d04-bbaf-58d1-906d-c2d459999c1b,FINAL,overall result,FINAL,365,Completed,,Final Result,


In [341]:
def getOverall(eventId, rallyId, stageId, championship="wrc", group="all"):
    stub = f"result/stageResult?eventId={eventId}&rallyId={rallyId}&stageId={stageId}&championship={championship}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_overall = tablify(json_data)
    stage_id_annotations(df_overall, eventId, rallyId, stageId)
    if group=="all":
        return df_overall
    else:
        return df_overall[df_overall["groupClass"]==group]

stageId = df_stageDetails[df_stageDetails["stageId"]!="FINAL"].iloc[-1]["stageId"]

getOverall(eventId, rallyId, stageId)#.head(3)

,id,pos,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,...,penaltyTime,time,totalTime,diffFirst,diffPrev,groupClass,championshipId,eventId,rallyId,stageId
0,79c1931c-0cab-5b80-bdaa-d8674c15997e,1,#33,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,Elfyn EVANS,53e56691-fe7c-5271-9dc5-8960df28b221,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,...,,3:32:08.8,3:32:08.8,,,RC1,None,365,369,6140
1,c58f0ad5-6dc9-5f3a-ab24-6d3306c89e06,2,#17,1cf9ade6-25b5-5586-8393-eacfeb943eae,France,Flags/FRA.png,Sébastien OGIER,867cd58f-3fe3-5290-9cc5-b21cb41c523c,France,Flags/FRA.png,...,1:00.0,3:33:26.5,3:33:26.5,+1:17.7,+1:17.7,RC1,None,365,369,6140
2,dee62ffc-9830-5564-97fd-b85d47ce61d2,3,#69,d8e4bbea-3af2-5486-9ad5-a445aaec573e,Finland,Flags/FIN.png,Kalle ROVANPERÄ,e4dd8a3f-00e9-59f7-9871-9337af6085d7,Finland,Flags/FIN.png,...,,3:33:55.3,3:33:55.3,+1:46.5,+28.8,RC1,None,365,369,6140
3,f2f3da23-d724-52a9-bd01-362e0e389e81,4,#4,916f5b10-fee9-5b4c-b17b-6fbbc343cc3c,Finland,Flags/FIN.png,Esapekka LAPPI,2ef28d31-4a74-5794-b720-12c327c941b6,Finland,Flags/FIN.png,...,,3:34:59.1,3:34:59.1,+2:50.3,+1:03.8,RC1,None,365,369,6140
4,4965f409-4d74-52b2-9a6b-c97db36bd1e7,5,#18,298f93b1-b0ef-5af4-9f0c-e468d29abfd2,Japan,Flags/JPN.png,Takamoto KATSUTA,6347a621-eade-5fff-a7b4-7c4b73651e8a,Ireland,Flags/IRL.png,...,1:10.0,3:35:19.1,3:35:19.1,+3:10.3,+20.0,RC1,None,365,369,6140
5,e3b0a0d6-ffcd-597d-8810-c9cc5e31cc13,6,#8,6632e7ca-34bf-55b8-9cad-d060000fa794,Estonia,Flags/EST.png,Ott TÄNAK,00a8a5c3-f7ba-5086-86df-1a59b7da7e26,Estonia,Flags/EST.png,...,,3:35:37.1,3:35:37.1,+3:28.3,+18.0,RC1,None,365,369,6140
6,a0977e64-2959-5722-8be1-ab4ea8573a75,7,#20,093f0b06-7a05-5353-92ac-f5a68ae51f41,Norway,Flags/NOR.png,Andreas MIKKELSEN,5d0fb669-fb61-5d51-805b-d09c1ac7e36a,Norway,Flags/NOR.png,...,,3:39:42.5,3:39:42.5,+7:33.7,+4:05.4,RC2,None,365,369,6140
7,795c4177-f68a-57a3-ad17-d4d2851e1dbe,8,#21,199a29d0-97f5-530f-9a5d-7ab45b1da0cc,ANA,Flags/ANA.png,Nikolay GRYAZIN,7d63ec40-0bb6-5da3-954e-846a2d40a9a1,ANA,Flags/ANA.png,...,,3:40:58.4,3:40:58.4,+8:49.6,+1:15.9,RC2,None,365,369,6140
8,c70c9ad6-e0bc-5ed7-a339-c2ace0b02885,9,#22,cede2cae-3446-54d8-b9d7-7e65651ebfd1,Poland,Flags/POL.png,Kajetan KAJETANOWICZ,a488aa5c-affc-5f48-a95e-0f400495fd71,Poland,Flags/POL.png,...,,3:51:34.7,3:51:34.7,+19:25.9,+10:36.3,RC2,None,365,369,6140
9,816f6cfd-592c-5b25-88a6-516a7b851c68,10,#35,e00988f4-2f7f-55f1-8ec6-a0e790578911,Japan,Flags/JPN.png,Hiroki ARAI,4f879f5d-2cce-5127-bf65-eb4034aee5f6,Japan,Flags/JPN.png,...,,3:54:31.5,3:54:31.5,+22:22.7,+2:56.8,RC4,None,365,369,6140


In [342]:
getOverall(eventId, rallyId, stageId).columns

Index(['id', 'pos', 'carNo', 'driverId', 'driverCountry', 'driverCountryImage',
       'driver', 'coDriverId', 'coDriverCountry', 'coDriverCountryImage',
       'coDriver', 'teamId', 'team/car', 'teamName', 'teamLogo', 'eligibility',
       'stageTime', 'penaltyTime', 'time', 'totalTime', 'diffFirst',
       'diffPrev', 'groupClass', 'championshipId', 'eventId', 'rallyId',
       'stageId'],
      dtype='object')

In [343]:
def getStageTimes(eventId, rallyId, stageId, championship="wrc"):
    stub = f"result/stageTimes?eventId={eventId}&rallyId={rallyId}&stageId={stageId}&championship={championship}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_stageTimes = tablify(json_data)
    stage_id_annotations(df_stageTimes, eventId, rallyId, stageId)
    return df_stageTimes


getStageTimes(eventId, rallyId, stageId).head(3)

,id,pos,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,...,teamLogo,eligibility,status,stageId,diffFirst,diffPrev,stageTime,championshipId,eventId,rallyId
0,f0e87af5-0cfa-5c94-a792-6106abeacde9,1,#11,c99a2a26-bd03-5153-aaa7-684d3acb5491,Belgium,Flags/BEL.png,Thierry NEUVILLE,b1b98699-0332-528a-8a80-11eb538f1ded,Belgium,Flags/BEL.png,...,teamLogo/hyundai.png,M,Completed,6140,,,4:48.8,None,365,369
1,e3b0a0d6-ffcd-597d-8810-c9cc5e31cc13,2,#8,6632e7ca-34bf-55b8-9cad-d060000fa794,Estonia,Flags/EST.png,Ott TÄNAK,00a8a5c3-f7ba-5086-86df-1a59b7da7e26,Estonia,Flags/EST.png,...,teamLogo/ford.png,M,Completed,6140,+2.4,+2.4,4:51.2,None,365,369
2,f2f3da23-d724-52a9-bd01-362e0e389e81,3,#4,916f5b10-fee9-5b4c-b17b-6fbbc343cc3c,Finland,Flags/FIN.png,Esapekka LAPPI,2ef28d31-4a74-5794-b720-12c327c941b6,Finland,Flags/FIN.png,...,teamLogo/hyundai.png,M,Completed,6140,+6.8,+4.4,4:55.6,None,365,369


In [344]:
#getStageTimes(365, 369, 4161).head(3)

In [345]:
def getSplitTimes(eventId, rallyId, stageId, championship="wrc"):
    stub = f"result/splitTime?eventId={eventId}&rallyId={rallyId}&stageId={stageId}&championship={championship}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_splitTimes = tablify(json_data)
    stage_id_annotations(df_splitTimes, eventId, rallyId, stageId)
    return df_splitTimes


getSplitTimes(eventId, rallyId, stageId).head(3)

,id,pos,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,...,splitPointTimeId,groupClass,start,stageTime,diffFirst,round1,round2,eventId,rallyId,stageId
0,151574c6-4998-59df-8c23-72c6fc8fcc26,1,#26,77222f77-61d8-5908-9632-24abf8556af7,Japan,Flags/JPN.png,Osamu FUKUNAGA,037327bb-b1e5-5db9-9f35-97747020724c,Japan,Flags/JPN.png,...,348824,RC2,2023-11-19T14:19:00+09:00,5:37.1,,3:58.7,5:05.7,365,369,6140
1,86e4c829-8070-555e-b509-dfd5877c8550,2,#22,cede2cae-3446-54d8-b9d7-7e65651ebfd1,Poland,Flags/POL.png,Kajetan KAJETANOWICZ,a488aa5c-affc-5f48-a95e-0f400495fd71,Poland,Flags/POL.png,...,348826,RC2,2023-11-19T14:23:00+09:00,5:18.6,-18.5,-12.4,-16.8,365,369,6140
2,68ee0f32-975d-59df-812d-138d26836c90,3,#25,209ebf34-4d0f-528b-83a2-46c38ceb499d,Poland,Flags/POL.png,Daniel CHWIST,ff1f11ca-f1dd-5b29-a16d-0af9510aa46c,Poland,Flags/POL.png,...,348828,RC2,2023-11-19T14:27:00+09:00,5:40.3,+3.2,+3.1,+2.5,365,369,6140


In [346]:
getSplitTimes(eventId, rallyId, stageId).columns

Index(['id', 'pos', 'carNo', 'driverId', 'driverCountry', 'driverCountryImage',
       'driver', 'coDriverId', 'coDriverCountry', 'coDriverCountryImage',
       'coDriver', 'teamId', 'team/car', 'teamName', 'teamLogo', 'eligibility',
       'splitPointId', 'splitPointTimeId', 'groupClass', 'start', 'stageTime',
       'diffFirst', 'round1', 'round2', 'eventId', 'rallyId', 'stageId'],
      dtype='object')

In [347]:
championshipId

247

In [348]:
def getStageWinners(eventId, championship="wrc"):
    championshipId = championshipIds[championship]
    stub = f"result/stageWinners?eventId={eventId}&championshipId={championshipId}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_stageWinners = tablify(json_data)
    return df_stageWinners


getStageWinners(eventId, ).head(3)

""


In [349]:
def getItinerary(eventId):
    stub = f"result/itinerary?eventId={eventId}&extended=true"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_itinerary = tablify(json_data, "values")
    return df_itinerary


getItinerary(eventId).tail(3)

,stage,eventId,stageId,distance,timingPrecision,firstCarDueDateTime,firstCarDueDateTimeMs,controlPenalties,status,type,location,targetDuration,targetDurationMs,id,order,date
23,SS22,365,4a0db031-0143-5756-af1f-e87945227001,7.52 km,Minute,14:15,2023-11-19T14:15:00+09:00,None,Completed,StageStart,Wolf Power Stage Asahi Kougen 2,00:03:00,180000,1095.0,4.0,Sunday 19th November
24,SF22,365,4a0db031-0143-5756-af1f-e87945227001,,Thousandth,,,None,Completed,FlyingFinish,Wolf Power Stage Asahi Kougen 2,00:00:00,0,1095.0,4.0,Sunday 19th November
25,TC22A,365,null,39.46 km,Minute,16:00,2023-11-19T16:00:00+09:00,Late,Running,TimeControl,FINISH - Technical Zone & Podium Holding Area IN,01:45:00,6300000,1095.0,4.0,Sunday 19th November


In [350]:
def getStartlist(eventId):
    stub = f"result/startLists?eventId={eventId}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_startlist = tablify(json_data, "startListItems")
    return df_startlist


getStartlist(eventId).tail(3)

,order,startDateTimeLocal,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,coDriver,teamId,team/car,teamName,teamLogo,eligibility,groupClass,priority,id,date
29,30,2023-11-19T05:34:00+09:00,#45,3ab588df-cc06-5017-8ba6-143de81a5972,Japan,Flags/JPN.png,Koji TAKATA,a655075d-d4b8-5790-b96c-ed6480de882c,Japan,Flags/JPN.png,Takashi SEKIMOTO,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,GR Yaris,Toyota,teamLogo/toyota.png,,JRCar1,None,d5126c53-c54e-514f-8b16-16ff6b9b1050,Sunday 2023-11-19
30,31,2023-11-19T05:34:00+09:00,#47,8584c859-11be-5de8-9ba1-f91976ec0b46,Japan,Flags/JPN.png,Satoshi IRIE,7336f886-56da-5a98-b9a8-fc626172e583,Japan,Flags/JPN.png,Satoshi KAGEYAMA,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Vitz,Toyota,teamLogo/toyota.png,,JRCar3,None,d5126c53-c54e-514f-8b16-16ff6b9b1050,Sunday 2023-11-19
31,32,2023-11-19T05:34:00+09:00,#48,4f12d0bb-2cbd-586a-86df-0844b79ef0d3,Japan,Flags/JPN.png,Hiroyasu SHIMIZU,bba42333-aeba-5102-8f37-191cf8de0723,Japan,Flags/JPN.png,Takahiro YASUI,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Yaris,Toyota,teamLogo/toyota.png,,JRCar3,None,d5126c53-c54e-514f-8b16-16ff6b9b1050,Sunday 2023-11-19


In [351]:
def getPenalties(eventId):
    stub = f"result/penalty?eventId={eventId}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_penalties = tablify(json_data)
    return df_penalties


getPenalties(eventId).head(3)

,id,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,coDriver,...,eligibility,groupClass,rallyId,entryId,controlId,penaltyTime,penaltyDuration,penaltyDurationMs,reason,control
0,4844,#46,e2464489-5926-5ac0-94a3-bdbf823bb827,Japan,Flags/JPN.png,Yasunori HAGIWARA,e6f37ce0-7731-5942-996d-63cde99f1e23,Japan,Flags/JPN.png,Koichi HARADA,...,,JRCar1,369,47504,25793,40.0,PT40S,40000,4 MINS LATE,TC1
1,4845,#34,d6030311-c622-5262-8a0e-8fb4e30bcc29,Japan,Flags/JPN.png,Toshi ARAI,05b11bf7-a75c-53e7-87eb-9ea080333b1c,Japan,Flags/JPN.png,Yuichi MATSUMOTO,...,,JRCar1,369,47494,25765,10.0,PT10S,10000,1 MIN LATE,TC1D
2,4846,#34,d6030311-c622-5262-8a0e-8fb4e30bcc29,Japan,Flags/JPN.png,Toshi ARAI,05b11bf7-a75c-53e7-87eb-9ea080333b1c,Japan,Flags/JPN.png,Yuichi MATSUMOTO,...,,JRCar1,369,47494,25765,10.0,PT10S,10000,1 MIN LATE,TC1D


In [314]:
def getRetirements(eventId):
    stub = f"result/retirements?eventId={eventId}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_retirements = tablify(json_data)
    return df_retirements


getRetirements(eventId).head(3)

,id,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,coDriver,...,team/car,teamName,teamLogo,eligibility,groupClass,rallyId,entryId,controlId,reason,control
0,8817,#44,0a4d6b0a-f926-56f3-aca2-049f93add127,Japan,Flags/JPN.png,Tomoyuki SHINKAI,8e4b14a8-433b-5001-b586-0f5cf15edf1f,Japan,Flags/JPN.png,Yuichi ANDO,...,GR Yaris,Toyota,teamLogo/toyota.png,,JR1,490,54575,34603,MECHANICAL,TC2
1,8818,#51,53f655b1-91e6-5163-adad-f8e7e4c6a77b,Japan,Flags/JPN.png,Mitsuhiro KUNISAWA,b57311ff-0e08-5f28-8806-efb80bfd6356,Japan,Flags/JPN.png,Masahiko KIHARA,...,Clio RS Line,Renault,teamLogo/renault.png,,RC5,490,54582,34605,MECHANICAL,SF2
2,8819,#38,b487d175-4d7a-563c-bf10-8ec3dc5f0bc3,Japan,Flags/JPN.png,Satoshi IMAI,ce5d6a41-d103-51fe-8b44-2cb3776f8831,New Zealand,Flags/NZL.png,Jason FARMER,...,C3,Citroen,teamLogo/citroen.png,WRC2 (DCM/CC),RC2,490,54570,34608,ACCIDENT,SF3


In [315]:
def getChampionship(
    seasonId, eventId, championship_type="driver", championship="all", retUrl=False
):
    """ championship_tpe: driver | codriver | manufacturer"""
    championshipId = championshipIds[championship]
    stub = f"result/championshipresult?seasonId={seasonId}&championshipId={championshipId}&type={championship_type}&championship={championship}"
    if retUrl:
        return stub
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_splitTimes = tablify(json_data)
    return df_splitTimes


championship_type = "driver"  # where is list?

getChampionship(seasonId, championship_type).head(3)

,championshipEntryId,overallPosition,overallPoints,driver,driverCountryImage,driverId,tyreManufacturer,teamName,teamLogo,_contry_iso_MCO,...,pointsBreakDown_contry_iso_EUR,position_contry_iso_EUR,status_contry_iso_EUR,totalPoints_contry_iso_EUR,_contry_iso_JPN,dropped_contry_iso_JPN,pointsBreakDown_contry_iso_JPN,position_contry_iso_JPN,status_contry_iso_JPN,totalPoints_contry_iso_JPN
0,5144,1,242,Thierry NEUVILLE,Flags/BEL.png,c99a2a26-bd03-5153-aaa7-684d3acb5491,None,Hyundai,teamLogo/hyundai.png,,...,13 + 3 + 2,(3),Finished,18,,false,6 + 7 + 4,(6),Finished,17
1,5146,2,210,Elfyn EVANS,Flags/GBR.png,ae7329c9-79b3-5d96-886c-22cca6217764,None,Toyota,teamLogo/toyota.png,,...,15 + 6 + 3,(2),Finished,24,,false,18 + 4 + 3,(1),Finished,25
2,5147,3,200,Ott TÄNAK,Flags/EST.png,6632e7ca-34bf-55b8-9cad-d060000fa794,None,Hyundai,teamLogo/hyundai.png,,...,18 + 4,(1),Finished,22,,false,R,(R),RetiredAccident,0


In [316]:
getChampionship(seasonId, eventId, championship_type)
#championshipresult?seasonId=28&championshipId=245&type=driver&championship=wrc

,championshipEntryId,overallPosition,overallPoints,driver,driverCountryImage,driverId,tyreManufacturer,teamName,teamLogo,_contry_iso_MCO,...,pointsBreakDown_contry_iso_EUR,position_contry_iso_EUR,status_contry_iso_EUR,totalPoints_contry_iso_EUR,_contry_iso_JPN,dropped_contry_iso_JPN,pointsBreakDown_contry_iso_JPN,position_contry_iso_JPN,status_contry_iso_JPN,totalPoints_contry_iso_JPN
0,5144,1,242,Thierry NEUVILLE,Flags/BEL.png,c99a2a26-bd03-5153-aaa7-684d3acb5491,None,Hyundai,teamLogo/hyundai.png,,...,13 + 3 + 2,(3),Finished,18,,false,6 + 7 + 4,(6),Finished,17
1,5146,2,210,Elfyn EVANS,Flags/GBR.png,ae7329c9-79b3-5d96-886c-22cca6217764,None,Toyota,teamLogo/toyota.png,,...,15 + 6 + 3,(2),Finished,24,,false,18 + 4 + 3,(1),Finished,25
2,5147,3,200,Ott TÄNAK,Flags/EST.png,6632e7ca-34bf-55b8-9cad-d060000fa794,None,Hyundai,teamLogo/hyundai.png,,...,18 + 4,(1),Finished,22,,false,R,(R),RetiredAccident,0
3,5145,4,191,Sébastien OGIER,Flags/FRA.png,1cf9ade6-25b5-5586-8393-eacfeb943eae,None,Toyota,teamLogo/toyota.png,,...,R,(R),RetiredAccident,0,,false,15 + 5 + 5,(2),Finished,25
4,5148,5,162,Adrien FOURMAUX,Flags/FRA.png,bf956e8a-5cad-5327-add0-26e0481ea508,None,Ford,teamLogo/ford.png,,...,0 + 5 + 1,(32),Finished,6,,false,13 + 3,(3),Finished,16
5,5150,6,116,Takamoto KATSUTA,Flags/JPN.png,298f93b1-b0ef-5af4-9f0c-e468d29abfd2,None,Toyota,teamLogo/toyota.png,,...,10 + 7 + 5,(4),Finished,22,,false,10 + 2 + 2,(4),Finished,14
6,5273,7,114,Kalle ROVANPERÄ,Flags/FIN.png,d8e4bbea-3af2-5486-9ad5-a445aaec573e,None,Toyota,teamLogo/toyota.png,,...,-,(-),DidNotEnter,0,,false,-,(-),DidNotEnter,0
7,5154,8,46,Grégoire MUNSTER,Flags/LUX.png,c718f17c-f6a0-54b5-bad9-a406b8cc95f0,None,Ford,teamLogo/ford.png,,...,8 + 2,(5),Finished,10,,false,8 + 1,(5),Finished,9
8,5870,9,44,Daniel SORDO,Flags/ESP.png,d06b6c54-863a-533d-9241-9dd04f7acb85,None,Hyundai,teamLogo/hyundai.png,,...,-,(-),DidNotEnter,0,,false,-,(-),DidNotEnter,0
9,5276,10,44,Sami PAJARI,Flags/FIN.png,caeac19e-6be8-509e-882a-28ff3a15af6a,None,Toyota,teamLogo/toyota.png,,...,R,(R),RetiredAccident,0,,false,3 + 0,(8),Finished,3


In [317]:
import sqlite_utils
#!rm wrc_2024_archive_results.db
_db_filepath = f"wrc_{CURRENT_YEAR}_archive_results.db"
pathlib.Path.unlink(_db_filepath) 
# db = sqlite_utils.Database("wrc_2023_results.db")

#!rm wrc_2024_12_results.db
db = sqlite_utils.Database(_db_filepath)

In [318]:
# db["delme"].insert_all(df_fullcal.to_dict("records"))
# db["delme"].drop()

In [319]:
def db_add(table, df):
    if not df.empty:
        db[table].insert_all(df.to_dict("records"), alter=True)

db_add("fullcal", getFullCalendar())
db_add("resultscal", getResultsCalendar())

In [320]:
# We can add alter=True to dynamically modify tables

def stageresults(row, eventId, rallyId):
    stageId = row["stageId"]
    if stageId=="FINAL":
        return
    try:
        _df_stageTimes = getStageTimes(eventId, rallyId, stageId)
        if 'event' in _df_stageTimes.columns:
            db_add("shakedown_stagetimes", _df_stageTimes)   
        else:
            db_add("stagetimes", _df_stageTimes)
    except:
        print(f"Couldn't get stagetimes for {eventId} {rallyId} {stageId}")
    try:
        _df_splitTimes = getSplitTimes(eventId, rallyId, stageId)
        if 'event' in _df_splitTimes.columns:
            db_add("shakedown_split", _df_splitTimes)   
        else:
            db_add("splittimes", _df_splitTimes)
    except:
          print(f"Couldn't get splittimes for {eventId} {rallyId} {stageId}")
    try:
        _df_Overall = getOverall(eventId, rallyId, stageId)
        if 'event' in _df_Overall.columns:
            db_add("shakedown_overall", _df_Overall)   
        else:
            db_add("overall", _df_Overall)
    except:
        print(f"Couldn't get overall for {eventId} {rallyId} {stageId}")


def rallyresults(row):
    eventId, rallyId = row[["eventId", "rallyId"]]
    db_add("retirements",getRetirements(eventId))
    db_add("penalties",getPenalties(eventId))
    db_add("startlist",getStartlist(eventId))
    db_add("itinerary",getItinerary(eventId))
    db_add("stagewinners",getStageWinners(eventId))

    df_stageDetails = getStageDetails(eventId, rallyId)
    db_add("stagedetails", df_stageDetails)
    df_stageDetails.apply(stageresults, axis=1, args=(eventId, rallyId))
    #stageresults(df_stageDetails)


def rallyresults2(row):
    eventId, rallyId = row[["eventId", "rallyId"]]
    print(eventId, rallyId, row["startDateLocal"])

_ = df_fullcal.apply(rallyresults, axis=1)

In [321]:
_ = df_fullcal.apply(rallyresults2, axis=1)

446 477 2024-01-24T16:30:00+01:00
447 478 2024-02-15T09:00:00+01:00
448 479 2024-03-27T10:00:00+03:00
449 480 2024-04-18T09:00:00+02:00
450 481 2024-05-09T08:00:00+01:00
452 483 2024-05-30T18:45:00+02:00
453 484 2024-06-27T10:00:00+02:00
454 485 2024-07-18T07:31:00+03:00
455 486 2024-07-31T19:00:00+03:00
456 487 2024-09-05T09:01:00+03:00
457 488 2024-09-26T08:31:00-03:00
458 489 2024-10-17T09:30:00+02:00
459 490 2024-11-21T09:00:00+09:00


In [95]:
df_fullcal.columns

Index(['id', 'guid', 'title', 'location', 'startDate', 'endDate', 'eventId',
       'rallyId', 'description', 'round', 'cvpSeriesLink', 'sponsor', 'images',
       'season', 'competition', 'country', 'asset', '__typename', 'type',
       'uid', 'seriesUid', 'releaseYear', 'availableOn', 'availableTill',
       'startDateLocal', 'endDateLocal', 'finishDate', 'championship',
       'championshipLogo'],
      dtype='object')

In [96]:
getStageTimes(eventId, rallyId, stageId)

,id,pos,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,...,team/car,teamName,teamLogo,eligibility,status,stageId,diffFirst,diffPrev,stageTime,championshipId
0,f0e87af5-0cfa-5c94-a792-6106abeacde9,1,#11,c99a2a26-bd03-5153-aaa7-684d3acb5491,Belgium,Flags/BEL.png,Thierry NEUVILLE,b1b98699-0332-528a-8a80-11eb538f1ded,Belgium,Flags/BEL.png,...,i20 N Rally1 HYBRID,Hyundai,teamLogo/hyundai.png,M,Completed,6140,,,4:48.8,None
1,e3b0a0d6-ffcd-597d-8810-c9cc5e31cc13,2,#8,6632e7ca-34bf-55b8-9cad-d060000fa794,Estonia,Flags/EST.png,Ott TÄNAK,00a8a5c3-f7ba-5086-86df-1a59b7da7e26,Estonia,Flags/EST.png,...,Puma Rally1 HYBRID,Ford,teamLogo/ford.png,M,Completed,6140,+2.4,+2.4,4:51.2,None
2,f2f3da23-d724-52a9-bd01-362e0e389e81,3,#4,916f5b10-fee9-5b4c-b17b-6fbbc343cc3c,Finland,Flags/FIN.png,Esapekka LAPPI,2ef28d31-4a74-5794-b720-12c327c941b6,Finland,Flags/FIN.png,...,i20 N Rally1 HYBRID,Hyundai,teamLogo/hyundai.png,M,Completed,6140,+6.8,+4.4,4:55.6,None
3,4965f409-4d74-52b2-9a6b-c97db36bd1e7,4,#18,298f93b1-b0ef-5af4-9f0c-e468d29abfd2,Japan,Flags/JPN.png,Takamoto KATSUTA,6347a621-eade-5fff-a7b4-7c4b73651e8a,Ireland,Flags/IRL.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,,Completed,6140,+7.1,+0.2,4:55.9,None
4,c58f0ad5-6dc9-5f3a-ab24-6d3306c89e06,5,#17,1cf9ade6-25b5-5586-8393-eacfeb943eae,France,Flags/FRA.png,Sébastien OGIER,867cd58f-3fe3-5290-9cc5-b21cb41c523c,France,Flags/FRA.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,Completed,6140,+7.3,+0.2,4:56.1,None
5,dee62ffc-9830-5564-97fd-b85d47ce61d2,6,#69,d8e4bbea-3af2-5486-9ad5-a445aaec573e,Finland,Flags/FIN.png,Kalle ROVANPERÄ,e4dd8a3f-00e9-59f7-9871-9337af6085d7,Finland,Flags/FIN.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,Completed,6140,+9.9,+2.6,4:58.7,None
6,79c1931c-0cab-5b80-bdaa-d8674c15997e,7,#33,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,Elfyn EVANS,53e56691-fe7c-5271-9dc5-8960df28b221,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,Completed,6140,+13.4,+3.5,5:02.2,None
7,795c4177-f68a-57a3-ad17-d4d2851e1dbe,8,#21,199a29d0-97f5-530f-9a5d-7ab45b1da0cc,ANA,Flags/ANA.png,Nikolay GRYAZIN,7d63ec40-0bb6-5da3-954e-846a2d40a9a1,ANA,Flags/ANA.png,...,Fabia RS,Skoda,teamLogo/skoda.png,WRC2 (T/DC/CC),Completed,6140,+21.4,+8.1,5:10.2,None
8,c70c9ad6-e0bc-5ed7-a339-c2ace0b02885,9,#22,cede2cae-3446-54d8-b9d7-7e65651ebfd1,Poland,Flags/POL.png,Kajetan KAJETANOWICZ,a488aa5c-affc-5f48-a95e-0f400495fd71,Poland,Flags/POL.png,...,Fabia RS,Skoda,teamLogo/skoda.png,WRC2 (DC/CC),Completed,6140,+29.7,+8.3,5:18.6,None
9,a0977e64-2959-5722-8be1-ab4ea8573a75,10,#20,093f0b06-7a05-5353-92ac-f5a68ae51f41,Norway,Flags/NOR.png,Andreas MIKKELSEN,5d0fb669-fb61-5d51-805b-d09c1ac7e36a,Norway,Flags/NOR.png,...,Fabia RS,Skoda,teamLogo/skoda.png,WRC2 (T/D),Completed,6140,+30.2,+0.5,5:19.0,None


In [97]:
for row in db.query("select DISTINCT(eventId) from stagedetails "):
    print(row)

{'eventId': '353'}
{'eventId': '354'}
{'eventId': '355'}
{'eventId': '356'}
{'eventId': '357'}
{'eventId': '358'}
{'eventId': '359'}
{'eventId': '360'}
{'eventId': '361'}
{'eventId': '362'}
{'eventId': '363'}
{'eventId': '364'}
{'eventId': '365'}


In [42]:
!ls -al wrc_2024_12_results.db

-rw-r--r--  1 tonyhirst  staff  3108864 27 Nov 00:07 wrc_2024_12_results.db


In [114]:
db.table_names()

['fullcal',
 'resultscal',
 'retirements',
 'penalties',
 'startlist',
 'itinerary',
 'stagedetails',
 'stagetimes',
 'splittimes',
 'overall']